# Scraping Bundesliga Data and Analyzing Data

In this project, we'll scrape data from the skysports website and generate a dataframe on a number of Bundesliga seasons. We'll make use of BeautifulSoup, Requests, and Pandas.

In each Bundesliga season, there are 34 games and 18 teams. A team earns 3 points for a win, 1 point for a draw, and 0 points for a loss.

## Importing Required Libraries

In [41]:
#let's begin by importing the required libraries
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import seaborn as sns

## Setting up to Scrape the Data
We're going to use an empty list (aptly) named 'soccer' to store a list of lists generated during the scrape. Each list within the list will contain one row of data, as well as an item that helps identify the season.

The URL convention used by SkySports ends in the year in which the season began. In order to loop through the seasons, we're going to create a list that contains the years that the data covers. We'll generate the list using the range() function and then convert those integers into strings. We'll store data from 2008 to 2018 (ignoring the partial 2019 season).

In [2]:
soccer = [] #create empty list to store data

#generate a list that contains those values
years_int = list(range(2008,2019))

#turn each value in that list into a string for concactenating to a URL later on using a list comprehension
years = [str(i) for i in years_int]

## Scraping the Data with a For Loop
We'll wrap our scrape in a for loop so that we can scrape multiple years' worth of data directly. The site stores the table body ('tbody') in a table class 'standing-table__table'. As the table data cells aren't uniquely classified, we generate another for loop that cycles through each table data (td) cell in a given table row (tr), and add a stripped version to the a list called 'row'. An empty variable is added to the list, which we re-purpose by filling in the season. Finally, we append the list 'row' to the list 'soccer'.

In [3]:
#create a loop that iterates through each year
for year in years:
    #get the html data
    url = 'https://www.skysports.com/bundesliga-table/'+ year #this follows the URL convention used by SkySports
    data = requests.get(url)

    #load data into soup variable
    soup = BeautifulSoup(data.text, 'html.parser')

    #find table and table data
    table_all = soup.find('table', {'class':'standing-table__table'})
    tablebody = table_all.find('tbody') 

    #create loop to go through each tr
    for row in tablebody.find_all('tr'):
        td = row.find_all('td')
        row = [i.text.strip() for i in td]
        row[-1] = year
        soccer.append(row)

## Generating a Pandas Dataframe
In order to manipulate the data and do some high-level analysis, we turn the list of lists ('soccer') into a dataframe called 'df'. We also assign column names. 

In [4]:
#generate a dataframe from the records
df = pd.DataFrame.from_records(soccer)
df.columns = ['Position', 'Team', 'Games Played', 'Won', 'Draw', 'Loss', 'Goals For', 'Goals Against', 'Goal Differential', 'Points', 'Season']

In [17]:
#let's make sure that our data is loaded correctly
df.head()

,Position,Team,Games Played,Won,Draw,Loss,Goals For,Goals Against,Goal Differential,Points,Season
0,1,Wolfsburg,34,21,6,7,80,41,39,69,2008
1,2,Bayern Munich,34,20,7,7,71,42,29,67,2008
2,3,Stuttgart,34,19,7,8,63,43,20,64,2008
3,4,Hertha Berlin,34,19,6,9,48,41,7,63,2008
4,5,Hamburg,34,19,4,11,49,47,2,61,2008


We can see that the data covers off the years that we want it to cover off and follows the same structure in each file. Let's begin doing some work in Pandas to dive into the data.

Let's make sure that our dataframe has correctly formatted values. We'll use the .info() function to identify the data type for each column.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 11 columns):
Position             198 non-null object
Team                 198 non-null object
Games Played         198 non-null object
Won                  198 non-null object
Draw                 198 non-null object
Loss                 198 non-null object
Goals For            198 non-null object
Goals Against        198 non-null object
Goal Differential    198 non-null object
Points               198 non-null object
Season               198 non-null object
dtypes: object(11)
memory usage: 17.1+ KB


We can see that the strings aren't formatted as strings and the integers aren't formatted as integers.

We can easily remedy this using the .astype() function

In [35]:
#let's format all the columns using a for loop and if statement. Since we only want the Teams column to be a 
#a string and all others as integers, we can create a simple loop based on df.columns

for column in df.columns:
    if column != 'Team':
        df[column] = df[column].astype(int)
    else:
        df[column] = df[column].astype(str)
        
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 11 columns):
Position             198 non-null int32
Team                 198 non-null object
Games Played         198 non-null int32
Won                  198 non-null int32
Draw                 198 non-null int32
Loss                 198 non-null int32
Goals For            198 non-null int32
Goals Against        198 non-null int32
Goal Differential    198 non-null int32
Points               198 non-null int32
Season               198 non-null int32
dtypes: int32(10), object(1)
memory usage: 9.4+ KB


Let's check to see if all team names are correct and fix any issues there may be.

In [60]:
df['Team'].unique()

array(['Wolfsburg', 'Bayern Munich', 'Stuttgart', 'Hertha Berlin',
       'Hamburg', 'Borussia Dortmund', 'Hoffenheim', 'Schalke',
       'Bayer Leverkusen', 'Werder Bremen', 'Hannover 96', 'Cologne',
       'Eintracht Frankfurt', 'Bochum', "M'gladbach", 'Energie Cottbus',
       'Karlsruher SC', 'Arminia Bielefeld', 'Mainz', 'SC Freiburg',
       'FC Nurnberg', 'Kaiserslautern', 'St Pauli', 'FC Augsburg',
       'Fortuna DÃ¼sseldorf', 'Greuther Furth', 'Eintracht Braunschweig',
       'SC Paderborn 07', 'FC Ingolstadt 04', 'SV Darmstadt 98',
       'RB Leipzig'], dtype=object)

We can see that Fortuna Dusselfdorf has some funny things going on, so let's replace those inline and re-print the list to make sure the substitution went as expected. (We'll ignore the umlaut to ensure proper printing).

In [118]:
df['Team'] = df['Team'].replace('Fortuna DÃ¼sseldorf', 'Fortuna Dusseldorf')
df['Team'].unique()

array(['Wolfsburg', 'Bayern Munich', 'Stuttgart', 'Hertha Berlin',
       'Hamburg', 'Borussia Dortmund', 'Hoffenheim', 'Schalke',
       'Bayer Leverkusen', 'Werder Bremen', 'Hannover 96', 'Cologne',
       'Eintracht Frankfurt', 'Bochum', "M'gladbach", 'Energie Cottbus',
       'Karlsruher SC', 'Arminia Bielefeld', 'Mainz', 'SC Freiburg',
       'FC Nurnberg', 'Kaiserslautern', 'St Pauli', 'FC Augsburg',
       'Fortuna Dusseldorf', 'Greuther Furth', 'Eintracht Braunschweig',
       'SC Paderborn 07', 'FC Ingolstadt 04', 'SV Darmstadt 98',
       'RB Leipzig'], dtype=object)

# Using Pandas for Analysis
Now that the columns are properly formatted and team names have been fixed, let's create some interesting insights!

Let's begin by looking at which team had the highest average points per season (note this will properly weigh averages for teams that haven't been in every season).

In [82]:
pivot = pd.pivot_table(df, index = 'Team', values = 'Points', aggfunc=np.mean)
pivot_sorted = pivot.sort_values('Points', ascending=False)
pivot_sorted

,Points
Team,
Bayern Munich,78.818182
Borussia Dortmund,66.181818
RB Leipzig,62.000000
Bayer Leverkusen,57.363636
Schalke,52.454545
Wolfsburg,49.363636
M'gladbach,48.727273
Hoffenheim,45.909091
Mainz,44.500000


## Number of Seasons per Team
Let's also create some analysis to see how many seasons each team has played in the league. To do this, we'll create a dictionary through a for loop that iterates over each team and increments a count by one. We'll then convert this dictionary into a dataframe to plot it out.

In [111]:
counts = {}

for team in df['Team']:
    if team not in counts:
        counts[team] = 1
    else:
        counts[team] += 1

seasons = pd.DataFrame(list(counts.items()))
seasons = seasons.rename(columns = {0:'Team', 1:'Seasons'})
seasons.sort_values('Seasons', ascending = False)


,Team,Seasons
0,Wolfsburg,11
6,Hoffenheim,11
1,Bayern Munich,11
14,M'gladbach,11
8,Bayer Leverkusen,11
7,Schalke,11
9,Werder Bremen,11
5,Borussia Dortmund,11
4,Hamburg,10
18,Mainz,10


## Goals, Wins, and Losses (oh my!)
Let's take a look at how teams perform generally in terms of how many goals, wins, and losses they get on average in a season.

In [119]:
pivot2 = pd.pivot_table(df, index = 'Team', values = ['Goal Differential', 'Won', 'Draw', 'Loss'], aggfunc = np.mean)
pivot2.sort_values('Goal Differential', ascending = False)

,Draw,Goal Differential,Loss,Won
Team,,,,
Bayern Munich,5.454545,57.181818,4.090909,24.454545
Borussia Dortmund,8.090909,32.272727,6.545455,19.363636
RB Leipzig,8.000000,21.666667,8.000000,18.000000
Bayer Leverkusen,8.000000,16.636364,9.545455,16.454545
Schalke,7.727273,8.454545,11.363636,14.909091
Wolfsburg,8.727273,4.454545,11.727273,13.545455
Hoffenheim,10.181818,2.636364,11.909091,11.909091
M'gladbach,7.818182,2.545455,12.545455,13.636364
Mainz,8.800000,-3.500000,13.300000,11.900000


Using BeautifulSoup and Pandas in Python we can do some really quick and easy analysis based on data spread across multiple webpages. Pandas makes it easy to turn data into useful insights.